In [ ]:
## Notebook for processing raw HVSMR data into HighRes volume and corresponding segmentations

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import glob
import numpy as np
import tensorflow as tf
import skimage
import nibabel as nib
from process_utils import *

In [ ]:
#Makes directory for HVSMR dataset

if not os.path.exists('./HVSMR'):
    os.makedirs('./HVSMR')

In [ ]:
#Loads images and segmentations
#Must put downloaded HVSMR data into './HVSMR' folder
def load_data_samples(pat_no, filename="./HVSMR/*.nii.gz"):
	clean_ims = []
	pix_dimensions = []
	if not os.path.exists("./HVSMR"):
		raise Exception("Error with file path.")
	else:
		files = glob.glob(filename)
		files = sorted(files)

		for i in range(1):
			pat = []
			for j in range(2):
       
				i=pat_no

				nii_img  = nib.load(files[i*3 + j])

				nii_data = nii_img.get_fdata()

				pat.append(np.transpose(nii_data[:,:,:],(2,1,0)))

			p1 = np.expand_dims(pat[0],-1)
			p2 = np.expand_dims(pat[1],-1)
			data = np.concatenate((p1,p2),-1)
			clean_ims.append(data)
			pix_dimensions.append(nii_img.header['pixdim'])
			
		return clean_ims, pix_dimensions


In [ ]:
#Makes directory to store data
if not os.path.exists('./seg_data'):
    os.makedirs('./seg_data')

#Set number of HVSMR datasets to process
HVSMR_No = 1

#Set output data size
z=112
x=256
y=128

#Loop for data generation
for pat in range(HVSMR_No):
    
	data, pix_dimensions_1 = load_data_samples(pat)

	segmentations = []
	for i in range(8):
		seg = (data[0][:,:,:,1]==(i+1))
		segmentations.append(seg)
	segmentations = np.array(segmentations)
	segmentations = np.transpose(segmentations,(1,2,3,0))
	im = np.expand_dims(norm(data[0][:,:,:,0]),-1)
	temp = np.concatenate((im,segmentations),-1)
	
	sf_x = pix_dimensions_1[0][3]
	sf_y = pix_dimensions_1[0][2]
	sf_z = pix_dimensions_1[0][1]

	temp = temp[::-1,:,:,:]

	#Rescale to 1.5mm isotropic
	rescaled_temp = []
	for channel in range(temp.shape[-1]):
		rescaled = skimage.transform.rescale(temp[...,channel], scale=[sf_x/1.5,sf_y/1.5,sf_z/1.5], order=3,anti_aliasing= True,preserve_range=True, mode ='constant',cval=0)
		rescaled_temp.append(rescaled)
  
	rescaled = np.array(rescaled_temp)
	rescaled = np.transpose(rescaled,(1,2,3,0))

	#Threshold to keep masks binary
	rounded_data = []
	for i in range(8):
		int_seg = rescaled[:,:,:,i+1] > 0.2
		rounded_data.append(int_seg)
	
	rounded_data = np.array(rounded_data)
	rounded_data = np.transpose(rounded_data,(1,2,3,0))
	temp = np.concatenate((rescaled[...,:1],rounded_data),-1)

	#Crop or pad to desired shape
	y_mid = find_com(temp[int(np.rint(temp.shape[0]/2)),:,:,0])[0]
	z_com = find_com(temp[:,:,int(y_mid),0])[1]
	cropped_vol= []
	seg_cropped_vol = []
	
	start = z_com - z/2
	empty_slice = np.zeros((256,128,8))
	empty_slice_1 = np.zeros((256,128,1))
	if start + z > temp.shape[0]:
		start = temp.shape[0]-z
		missing = 0
	if start < 0:
		start = 0
		missing = z - temp.shape[0]
	else:
		missing = 0

	if missing!=0:
		pad_number_1 = int(np.rint(missing/2))
		for j in range(pad_number_1):
			cropped_vol.append(empty_slice_1)
			seg_cropped_vol.append(empty_slice)
	for i in range(temp.shape[0]):
		slice = temp[i,...,:1]
		seg_slice = temp[i,...,1:]
		if start <= i < start+z:
			pad_x = x-slice.shape[0]
			pad_y = y - slice.shape[1]
			if pad_x > 0 and pad_y >0:
				slice = np.pad(slice,((int(np.floor(pad_x/2)),int(np.ceil(pad_x/2))),(int(np.floor(pad_y/2)),int(np.ceil(pad_y/2))),(0,0)),mode='symmetric')
				seg_slice = np.pad(seg_slice,((int(np.floor(pad_x/2)),int(np.ceil(pad_x/2))),(int(np.floor(pad_y/2)),int(np.ceil(pad_y/2))),(0,0)),mode='constant')
			elif pad_x > 0:
				slice = np.pad(slice,((int(np.floor(pad_x/2)),int(np.ceil(pad_x/2))),(0,0),(0,0)),mode='symmetric')
				seg_slice = np.pad(seg_slice,((int(np.floor(pad_x/2)),int(np.ceil(pad_x/2))),(0,0),(0,0)),mode='constant')
				slice = tf.image.resize_with_crop_or_pad(slice,x,y)
				seg_slice = tf.image.resize_with_crop_or_pad(seg_slice,x,y)
			elif pad_y > 0:
				slice = np.pad(slice,((0,0),(int(np.floor(pad_y/2)),int(np.ceil(pad_y/2))),(0,0)),mode='symmetric')
				seg_slice = np.pad(seg_slice,((0,0),(int(np.floor(pad_y/2)),int(np.ceil(pad_y/2))),(0,0)),mode='constant')
				slice = tf.image.resize_with_crop_or_pad(slice,x,y)
				seg_slice = tf.image.resize_with_crop_or_pad(seg_slice,x,y)
			else:
				slice = tf.image.resize_with_crop_or_pad(slice,x,y)
				seg_slice = tf.image.resize_with_crop_or_pad(seg_slice,x,y)
			cropped_vol.append(slice)
			seg_cropped_vol.append(seg_slice)
	if missing!=0:
		pad_number_2 = missing - pad_number_1
		for j in range(pad_number_2):
			cropped_vol.append(empty_slice_1)
			seg_cropped_vol.append(empty_slice)

	cropped_vol_1 = np.array(cropped_vol)
	seg_cropped_vol_1 = np.array(seg_cropped_vol)
 
	#Fill LV and RV masks
	new_masks = []
	for i in range(2):
		ch = seg_cropped_vol_1[...,i]
		ch = tf.expand_dims(tf.expand_dims(tf.cast(ch,tf.float32),0),-1)
		new_ch = fill_mask_3d(ch,iter=8)
		new_masks.append(new_ch[0,...])
	final = np.array(new_masks)
	final = np.transpose(final,(4,1,2,3,0))
 
	cropped_vol_1 = apply_clahe(cropped_vol_1) #CLAHE
	out = np.concatenate((cropped_vol_1[...,:1],final[0,...],seg_cropped_vol_1[...,3:4],seg_cropped_vol_1[...,4:5],seg_cropped_vol_1[...,5:6],seg_cropped_vol_1[...,6:7]),-1)

	np.save(f'./seg_data/High_Res_HVSMR_NoAug_{pat}.npy', out)